In [47]:
import pandas as pd
import numpy as np
import time
import os
import sys
import utils 
import itertools
import random as rand
import math
import copy
sys.path.append('../')
sys.path.insert(1, '../exp')

# sklearn
from sklearn.model_selection import cross_validate, train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn import metrics
from sklearn.feature_selection import mutual_info_classif
from sklearn.utils import compute_class_weight

import xgboost as xgb

#PostgreSQL
import psycopg2
from sqlalchemy import create_engine

# sqlgen
import optuna
from optuna import study
from optuna.trial import Trial as trial
from optuna.samplers import RandomSampler, TPESampler

# graph
import matplotlib.pyplot as plt
import plotly

# featuretools
import featuretools as ft

import warnings
warnings.filterwarnings('ignore')

## Step 1. Load data

In [48]:
path = "../exp_data/Tmall/"
train_data = pd.read_csv(os.path.join(path, "train_data.csv"))
test_data = pd.read_csv(os.path.join(path, "test_data.csv"))
user_log = pd.read_csv(os.path.join(path, "user_log.csv"))

train_data = train_data.drop(train_data.columns[0], axis=1)
train_data = train_data.drop(columns=['item_id', 'cat_id', 'brand_id', 'action_type'])
train_data = train_data.drop_duplicates()


test_data = test_data.drop(test_data.columns[0], axis=1)
test_data = test_data.drop(columns=['item_id', 'cat_id', 'brand_id', 'action_type'])
test_data = test_data.drop_duplicates()
#user_log = user_log.drop(user_log.columns[0], axis=1)
# data = pd.concat([train_data, test_data])
train_data['train_comb'] = train_data['user_id'].astype(str) + ',' + train_data['merchant_id'].astype(str)
test_data['test_comb'] = test_data['user_id'].astype(str) + ',' + test_data['merchant_id'].astype(str)
user_log['user_log_comb'] = user_log['user_id'].astype(str) + ',' + user_log['merchant_id'].astype(str)

# train_data = train_data.rename(columns={'Unnamed: 0':'train_index'})
# test_data = test_data.rename(columns={'Unnamed: 0':'test_index'})
user_log = user_log.rename(columns={'Unnamed: 0':'user_log_index'})
print(len(user_log))
print("Train true label:", len(train_data[train_data['label']==1]), 'Train false label:', len(train_data[train_data['label']==0]))
print("Test true label:", len(test_data[test_data['label']==1]), 'Test false label:', len(test_data[test_data['label']==0]))
train_data.head()

345699
Train true label: 751 Train false label: 795
Test true label: 205 Test false label: 192


,user_id,merchant_id,age_range,gender,label,train_comb
0,213820,2066,2,0,0,"213820,2066"
1,195,2286,5,0,1,"195,2286"
2,318509,4143,6,0,0,"318509,4143"
3,323850,361,6,0,1,"323850,361"
4,380935,3191,3,0,1,"380935,3191"


In [49]:
# user_info = pd.read_csv(os.path.join(path, "user_info_format1.csv"))
# train_data = pd.read_csv(os.path.join(path, "train_format1.csv"))
# train_data = train_data.merge(user_info, how='left')
# train_data = train_data.fillna(0)
# train_data.head()
# train_data['train_comb'] = train_data['user_id'].astype(str) + ',' + train_data['merchant_id'].astype(str)
# len(train_data)

In [50]:
len(train_data['train_comb'].unique())

1546

In [51]:
user_log.head()

,user_log_index,user_id,merchant_id,time_stamp,action_type,item_id,cat_id,brand_id,age_range,gender,user_log_comb
0,0,323837,3420,1024,0,8,1,12,0,0,"323837,3420"
1,1,323837,1393,1030,0,7,8,11,0,0,"323837,1393"
2,2,323837,1064,1030,0,4,0,12,0,0,"323837,1064"
3,3,323837,1064,1030,0,6,0,12,0,0,"323837,1064"
4,4,323837,1064,1030,2,4,0,12,0,0,"323837,1064"


## Step 2. Verify the performance of featuretools

In [52]:
# Verify Join all tables into one
new_train_data = pd.merge(user_log, train_data, how='left', left_on=['user_id','merchant_id'], right_on = ['user_id','merchant_id'])
new_train_data = new_train_data.loc[pd.notna(new_train_data['label'])]

new_train_labels = new_train_data['label']
new_train_data = new_train_data.drop(columns=['user_log_index', 'user_log_comb', 'train_comb', 'label'])

clf = XGBClassifier(random_state=0)

scores = cross_validate(
    clf,
    new_train_data,
    new_train_labels,
    cv=5,
    scoring='roc_auc',
    return_train_score=True,
    n_jobs=-1,
    return_estimator=True,
)
print(scores["test_score"].mean())

0.5628332865716374


In [53]:
new_train_data = pd.merge(user_log, train_data, how='left', left_on=['user_id','merchant_id'], right_on = ['user_id','merchant_id'])
new_train_data = new_train_data.loc[pd.notna(new_train_data['label'])]
new_train_data

,user_log_index,user_id,merchant_id,time_stamp,action_type,item_id,cat_id,brand_id,age_range_x,gender_x,user_log_comb,age_range_y,gender_y,label,train_comb
45,60,323837,4966,1110,0,8,9,8,0,0,"323837,4966",0.0,0.0,1.0,"323837,4966"
46,61,323837,4966,1110,0,9,5,8,0,0,"323837,4966",0.0,0.0,1.0,"323837,4966"
47,63,323837,4966,1110,0,17,4,8,0,0,"323837,4966",0.0,0.0,1.0,"323837,4966"
50,71,323837,4966,1110,0,12,2,8,0,0,"323837,4966",0.0,0.0,1.0,"323837,4966"
69,109,323837,4966,1111,2,8,9,8,0,0,"323837,4966",0.0,0.0,1.0,"323837,4966"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345646,538078,109074,4173,1111,0,10,2,13,3,1,"109074,4173",3.0,1.0,0.0,"109074,4173"
345647,538080,109074,4173,1111,0,5,6,13,3,1,"109074,4173",3.0,1.0,0.0,"109074,4173"
345656,538098,109074,4173,1111,0,3,4,13,3,1,"109074,4173",3.0,1.0,0.0,"109074,4173"
345669,538129,109074,4173,1110,0,13,6,13,3,1,"109074,4173",3.0,1.0,0.0,"109074,4173"


In [55]:
new_test_data = pd.merge(test_data, user_log, how='left', left_on=['user_id','merchant_id'], right_on = ['user_id','merchant_id'])
new_test_data

,user_id,merchant_id,age_range_x,gender_x,label,test_comb,user_log_index,time_stamp,action_type,item_id,cat_id,brand_id,age_range_y,gender_y,user_log_comb
0,71646,2537,2,0,0,"71646,2537",500361.0,914.0,0.0,6.0,2.0,14.0,2.0,0.0,"71646,2537"
1,71646,2537,2,0,0,"71646,2537",500602.0,1111.0,0.0,2.0,0.0,14.0,2.0,0.0,"71646,2537"
2,71646,2537,2,0,0,"71646,2537",500618.0,529.0,0.0,12.0,5.0,14.0,2.0,0.0,"71646,2537"
3,71646,2537,2,0,0,"71646,2537",500823.0,1111.0,2.0,2.0,0.0,14.0,2.0,0.0,"71646,2537"
4,71646,2537,2,0,0,"71646,2537",501020.0,911.0,0.0,5.0,5.0,14.0,2.0,0.0,"71646,2537"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9000,270861,3689,0,0,0,"270861,3689",71629.0,1109.0,0.0,16.0,4.0,1.0,0.0,0.0,"270861,3689"
9001,134842,4976,0,0,0,"134842,4976",140311.0,718.0,0.0,13.0,3.0,14.0,0.0,0.0,"134842,4976"
9002,134842,4976,0,0,0,"134842,4976",140321.0,1111.0,0.0,19.0,6.0,14.0,0.0,0.0,"134842,4976"
9003,134842,4976,0,0,0,"134842,4976",140323.0,1111.0,0.0,14.0,6.0,14.0,0.0,0.0,"134842,4976"


In [41]:
len(pd.concat([new_train_data['user_id'], new_train_data['merchant_id']]).unique())

2008

In [42]:
import featuretools as ft
train_labels = train_data['label']
train_data = train_data.drop(columns=['label'])
test_data = test_data['label']
test_data = test_data.drop(columns=['label'])

# dataframes = {
#     "train_data": (train_data, "train_comb"),
#     "user_log": (user_log, "user_log_index")
# }

# relationships = [
#     ("train_data", "train_comb", "user_log", "user_log_comb")
# ]
es = ft.EntitySet("tmall")
es.add_dataframe(
    dataframe_name="train_data",
    dataframe=train_data,
    index="train_comb"
)
es.add_dataframe(
    dataframe_name="user_log",
    dataframe=user_log,
    index="user_log_index"
)

es = es.add_relationship("train_data", "train_comb", "user_log", "user_log_comb")



In [43]:
start = time.time()
feature_matrix, features = ft.dfs(target_dataframe_name="train_data",
                                  agg_primitives = ["sum", "min", "max", "count", "mean"],
                                  #where_primitives = ["count"],
                                  #trans_primitives = [],
                                  ignore_columns={"train_data":['train_comb'],
                                                  "user_log":['user_log_index', 'user_log_comb']},
                                  entityset=es,
                                  verbose=True)
end = time.time()
print("Time:", end-start)

Built 40 features
Elapsed: 00:00 | Progress: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████
Time: 0.36020398139953613


In [44]:
feature_matrix.columns

Index(['user_id', 'merchant_id', 'age_range', 'gender',
       'MAX(user_log.action_type)', 'MAX(user_log.age_range)',
       'MAX(user_log.brand_id)', 'MAX(user_log.cat_id)',
       'MAX(user_log.gender)', 'MAX(user_log.item_id)',
       'MAX(user_log.merchant_id)', 'MAX(user_log.time_stamp)',
       'MAX(user_log.user_id)', 'MEAN(user_log.action_type)',
       'MEAN(user_log.age_range)', 'MEAN(user_log.brand_id)',
       'MEAN(user_log.cat_id)', 'MEAN(user_log.gender)',
       'MEAN(user_log.item_id)', 'MEAN(user_log.merchant_id)',
       'MEAN(user_log.time_stamp)', 'MEAN(user_log.user_id)',
       'MIN(user_log.action_type)', 'MIN(user_log.age_range)',
       'MIN(user_log.brand_id)', 'MIN(user_log.cat_id)',
       'MIN(user_log.gender)', 'MIN(user_log.item_id)',
       'MIN(user_log.merchant_id)', 'MIN(user_log.time_stamp)',
       'MIN(user_log.user_id)', 'SUM(user_log.action_type)',
       'SUM(user_log.age_range)', 'SUM(user_log.brand_id)',
       'SUM(user_log.cat_id)', 'SUM(u

In [45]:
from xgboost import XGBClassifier
clf = XGBClassifier(random_state=0)

scores = cross_validate(
    clf,
    feature_matrix,
    train_labels.to_frame(),
    cv=5,
    scoring='roc_auc',
    return_train_score=True,
    n_jobs=-1,
    return_estimator=True,
)
print(scores["test_score"].mean())

0.694529329279298


## Step 2. Store table in PostragSQL
Store train_data to database

In [ ]:
engine = utils.store_tmall(user_log)

{'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'auto_fg', 'host': '127.0.0.1', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 14.5 on x86_64-apple-darwin20.6.0, compiled by Apple clang version 12.0.0 (clang-1200.0.32.29), 64-bit',) 

tmall_log Table created successfully in PostgreSQL. 




## Step 3. Feature Generation & Machine Learning

In [ ]:
train_data.head()

In [ ]:
user_log.head()

In [29]:
#valid_score, clfs = utils.model_tmall(train_data)
y = train_data['label']
y = y.to_frame()
X = train_data.drop(['label'], axis=1)
X = X.fillna(0)

seeds = [0, 42, 158, 359, 1280]
test_scores = []
for seed in seeds:
    clf = xgb.XGBRegressor(random_state=0)
    clf.fit(X, y)
    test_pred = clf.predict(test_data.drop(columns='label', axis=1))
    test_score = metrics.roc_auc_score(test_data['label'], test_pred)
    test_scores.append(test_score)
print(sum(test_scores) / len(test_scores))

0.6572015224358975


In [30]:
global copy_train_data, copy_test_data, train_data, test_data
global trail_auc_store, trail_mi_store, train_auc_store

trail_auc_store = []
trail_mi_store = []
train_auc_store = []
copy_train_data = train_data
copy_test_data = test_data

global random_args_log, tpe_args_log
global random_user_train_log, tpe_user_train_log

random_args_log = {}
tpe_args_log = {}
random_user_train_log = {}
tpe_user_train_log = {}

global eva_global, seed_global

In [31]:
# res_x is to store top features 
global res_x 
res_x = list()

aggregation = ['SUM', 'MIN', 'MAX', 'COUNT', 'AVG'] 
m_attr = ['merchant_id', 'item_id', 'brand_id', 'cat_id']
categorical = ['gender', 'age_range', 'action_type']
numerical = ['time_stamp']
attributes = categorical + numerical

In [32]:
def update_user_train(args, train_data):  
    agg, m, lb_day, ub_day, w_cat_value_0, w_cat_value_1, w_cat_value_2 = args 
    agg = aggregation[agg]
    m = m_attr[m]
    
    w_cat_value_temp = [w_cat_value_0, w_cat_value_1, w_cat_value_2] #, w_cat_value_3]
    w_cat = []
    w_cat_value = []

    for i in range(len(categorical)):
        if w_cat_value_temp[i] != -1:
            w_cat.append(categorical[i])
            w_cat_value.append(w_cat_value_temp[i])
            
    sql_output = utils.generate_feature_in_small_space_tmall(engine, agg, m, lb_day, ub_day, w_cat, w_cat_value) 
    w_cat_str = "_".join(str(w_cat[x])+'='+str(w_cat_value[x]) for x in range(len(w_cat)))
    new_feature = pd.DataFrame(sql_output, columns = ['user_id', agg+'('+m+')_'+w_cat_str+'_'+'lb_day='+str(lb_day)+"_ub_day="+str(ub_day)]) #+'='+g_cat
    new_feature = new_feature.astype('float')
    new_train_data = train_data.merge(new_feature, how='left')
    
    return new_train_data, args

In [33]:
# define an objective function
def objective(trial):
    agg = trial.suggest_categorical('agg', np.array([i for i in range(len(aggregation))]))
    m = trial.suggest_categorical('m', np.array([i for i in range(len(m_attr))]))  
    lb_day = trial.suggest_categorical('lb_day', sorted_time[0: math.ceil(len(sorted_time)/2)])
    ub_day = trial.suggest_categorical('ub_day', sorted_time[math.ceil(len(sorted_time)/2)+1: -1])
    w_cat_value_0 = trial.suggest_categorical('w_cat_value_0', np.append(user_log[categorical[0]].unique(), [-1], 0))
    w_cat_value_1 = trial.suggest_categorical('w_cat_value_1', np.append(user_log[categorical[1]].unique(), [-1], 0))
    w_cat_value_2 = trial.suggest_categorical('w_cat_value_2', np.append(user_log[categorical[2]].unique(), [-1], 0))

    args = [agg, m, lb_day, ub_day, w_cat_value_0, w_cat_value_1, w_cat_value_2] #, w_cat_value_3]
    new_train_data, args = update_user_train(args, train_data)
    score, _ = utils.model_tmall(new_train_data)
    
    #是不是不应该这么粗暴地去重？？
    # for x in res_x:
    #     if args == x:
    #         score -= 0.2
    #         break
        
    print(args, " auc: ", score)
    return score

In [34]:
# define a NEW function with mutual information

def mi(trial):
    agg = trial.suggest_categorical('agg', np.array([i for i in range(len(aggregation))]))
    m = trial.suggest_categorical('m', np.array([i for i in range(len(m_attr))]))  
    lb_day = trial.suggest_categorical('lb_day', sorted_time[0: math.ceil(len(sorted_time)/2)])
    ub_day = trial.suggest_categorical('ub_day', sorted_time[math.ceil(len(sorted_time)/2)+1: -1])
    w_cat_value_0 = trial.suggest_categorical('w_cat_value_0', np.append(user_log[categorical[0]].unique(), [-1], 0))
    w_cat_value_1 = trial.suggest_categorical('w_cat_value_1', np.append(user_log[categorical[1]].unique(), [-1], 0))
    w_cat_value_2 = trial.suggest_categorical('w_cat_value_2', np.append(user_log[categorical[2]].unique(), [-1], 0))

    args = [agg, m, lb_day, ub_day, w_cat_value_0, w_cat_value_1, w_cat_value_2] #, w_cat_value_3]
    new_train_data, args = update_user_train(args, train_data)

    mi_matrix = mutual_info_classif(new_train_data.fillna(0), new_train_data['label'], random_state=0)
    mi_score = mi_matrix[-1]
    
    #是不是不应该这么粗暴地去重？？
    # for x in res_x:
    #     if args == x:
    #         mi_score = 0
    #         break
            
    print(args, "MI:", mi_score)
    return mi_score

In [35]:
# Random sampling 
def evaluate(trial_arr, mi_log, train_size):
    x_features = pd.DataFrame()
    x_features['index'] = range(0, len(mi_log))
    x_features['mi'] = mi_log
    x_features['agg'] = [trial_arr[i]['agg'] for i in range(len(trial_arr))]
    x_features['m'] = [trial_arr[i]['m'] for i in range(len(trial_arr))]
    x_features['lb_day'] = [trial_arr[i]['lb_day'] for i in range(len(trial_arr))]
    x_features['ub_day'] = [trial_arr[i]['ub_day'] for i in range(len(trial_arr))]
    x_features['w_cat_value_0'] = [trial_arr[i]['w_cat_value_0'] for i in range(len(trial_arr))]
    x_features['w_cat_value_1'] = [trial_arr[i]['w_cat_value_1'] for i in range(len(trial_arr))]
    x_features['w_cat_value_2'] = [trial_arr[i]['w_cat_value_2'] for i in range(len(trial_arr))]

    print(x_features.shape)
    x_train, x_test = train_test_split(x_features, train_size=200, random_state=0)
    
    y_acc = []
    for index, feature in x_train.iterrows():
        args = [int(feature['agg']),
               int(feature['m']),
               int(feature['lb_day']),
               int(feature['ub_day']),
               int(feature['w_cat_value_0']),
               int(feature['w_cat_value_1']),
               int(feature['w_cat_value_2'])] 
        new_train_data, args = update_user_train(args, train_data)
        
        y_acc.append(utils.model_tmall(new_train_data)[0])

    y_train = pd.DataFrame()
    y_train['index'] = x_train['index']
    y_train['label'] = y_acc

    global clf
    clf =  RandomForestRegressor(random_state=0) #LinearRegression()
    clf.fit(x_train[['mi']], y_train['label'])
    
    global estimated_accuracy
    estimated_accuracy = clf.predict(x_features[['mi']])
    predict_y = pd.DataFrame()
    predict_y['index'] = range(0, len(mi_log))
    predict_y['label'] = estimated_accuracy
    
    for index, row in y_train.iterrows():
        predict_y.loc[predict_y['index'] == row['index'], 'label'] = row['label']
    
    best_trail_index = y_train.nlargest(1, ['label']).index[0]
    train_auc_store.append(predict_y['label'].to_numpy())
    return predict_y['label'].to_numpy(), best_trail_index

In [36]:
top = 5

n_calls = [1500]
random_state_arr = [0] 

for n_call in n_calls:
    tpe_args_log[n_call] = {}
    tpe_user_train_log[n_call] = {}
    for seed in random_state_arr:
        tpe_args_log[n_call][seed] = []


# SQLGEN - Opt3

# Evaluate Test ROC-AUC

In [39]:
eva_store_optuna = []

start = time.time()
for eva in n_calls:
    print("trails:", eva)
    eva_global = eva
    seed_store = []
    for seed in random_state_arr:
        seed_global = seed
        global train_data, res_x 
        res_optuna_lst = []
        res_optuna_fun = []
        res_x = list()
        train_data = copy_train_data
        print("seed:", seed)

        for i in range(top):
            tpe_result = study.create_study(
                direction="maximize", 
                sampler=TPESampler(n_startup_trials=20, seed=seed)) #
            # tpe_result.optimize(objective, n_trials=eva, 
            #                     mi_initializer=mi, evaluate=evaluate, mi_init_trails=1000, train_size=100)
            tpe_result.optimize(objective, n_trials=eva)
            
            best_trial = []
            for key, value in tpe_result.best_trial.params.items():
                best_trial.append(value)
            res_x.append(best_trial)

            train_data, args = update_user_train(best_trial, train_data)
            res_optuna_lst.append(tpe_result.best_trial.params)
            res_optuna_fun.append(tpe_result.best_value)

            print(tpe_result.best_trial.params, tpe_result.best_value)
            print(train_data.shape)
            tpe_args_log[eva][seed].append(args)
        seed_store.append(res_optuna_fun)
        tpe_user_train_log[eva][seed] = train_data
        
    eva_store_optuna.append(seed_store)
end = time.time()
print("Time:", end-start)

[I 2022-09-21 21:57:24,033] A new study created in memory with name: no-name-39b460c9-7edc-4cfe-bda9-d4a393ade105
[W 2022-09-21 21:57:24,039] Trial 0 failed because of the following error: NameError("name 'sorted_time' is not defined")
Traceback (most recent call last):
  File "/Users/danruiqi/.pyenv/versions/3.9.6/lib/python3.9/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/mw/_z63l58x77q255c57knqd4dr0000gn/T/ipykernel_45313/1752518.py", line 5, in objective
    lb_day = trial.suggest_categorical('lb_day', sorted_time[0: math.ceil(len(sorted_time)/2)])
NameError: name 'sorted_time' is not defined


trails: 1500
seed: 0


NameError: name 'sorted_time' is not defined

In [ ]:
def model_tmall(user_train):
    y = user_train['label']
    y = y.to_frame()
    X = user_train.drop(['label'], axis=1)
    X = X.fillna(0)

    clf = xgb.XGBRegressor(random_state=0)
    auc = cross_validate(clf, X, y, cv=5,scoring=('roc_auc'), return_train_score=True, n_jobs=-1, return_estimator=True)
    valid_auc = auc['test_score'].mean()
    train_auc = auc['train_score'].mean()

    return train_auc, valid_auc, auc['estimator'][0]


In [ ]:
test_auc = {}
train_auc = {}
valid_auc = {}
for n_calls in tpe_args_log:
    for seed in tpe_args_log[n_calls]:
        test_auc[seed] = []
        train_auc[seed] = []
        valid_auc[seed] = []
        train_data = copy_train_data
        test_data = copy_test_data
        
        # Default AUC without new features
        train, valid, clf = model_tmall(train_data)
        train_auc[seed].append(train)
        valid_auc[seed].append(valid)
        test_pred = clf.predict(test_data.drop(columns='label', axis=1))
        test_auc[seed].append(metrics.roc_auc_score(test_data['label'], test_pred))
        for args in tpe_args_log[n_calls][seed]:
            test_data, config = update_user_train(args, test_data)
            train_data, config = update_user_train(args, train_data)
            train, valid, clf = model_tmall(train_data)
            train_auc[seed].append(train)
            valid_auc[seed].append(valid)
            
            test_data = test_data.fillna(0)
            test_pred = clf.predict(test_data.drop(columns='label', axis=1))
            test_auc[seed].append(metrics.roc_auc_score(test_data['label'], test_pred))
train_auc, valid_auc, test_auc